<h1>CS181 Practical 1</h1>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [2]:
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import DataStructs
from rdkit.Chem import MCS
from rdkit.Chem import Draw

In [3]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [4]:
df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2nsnc12,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.19
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[SiH2]C=c12,1,0,0,0,1,0,1,0,0,...,1,0,0,1,0,0,0,0,0,1.60
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-23)c2ccccc12,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.49
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13)c1=C[SiH2]C=c21,1,0,0,0,1,1,1,0,0,...,1,0,0,0,1,0,0,0,0,1.36
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1.98


In [5]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [6]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2nsnc12,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[SiH2]C=c12,1,0,0,0,1,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-23)c2ccccc12,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13)c1=C[SiH2]C=c21,1,0,0,0,1,1,1,0,0,...,0,1,0,0,0,1,0,0,0,0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [7]:
df_test = pd.DataFrame(df_all['smiles'])

In [12]:
hydrogen = []
carbon = []
oxygen = []
nitrogen = []
silicon = []
selenium = []
doublebonds = []

smile_list = df_test['smiles'].tolist()

In [14]:
for smile in smile_list:

    
    hydrogen_count = smile.count("h")
    hydrogen_count = hydrogen_count + smile.count("H")
    hydrogen.append(hydrogen_count)
    
    carbon_count = smile.count("c")
    carbon_count = carbon_count + smile.count("C")
    carbon.append(carbon_count)
    
    oxygen_count = smile.count("o")
    oxygen_count = oxygen_count + smile.count("O")
    oxygen.append(oxygen_count)
    
    nitrogen_count = smile.count("n")
    nitrogen_count = nitrogen_count + smile.count("N")
    nitrogen.append(nitrogen_count)
    
    silicon_count = smile.count("si")
    silicon_count = silicon_count + smile.count("Si")
    silicon.append(silicon_count)
    
    selenium_count = smile.count("se")
    selenium_count = selenium_count + smile.count("Se")
    selenium.append(selenium_count)
                    
    doublebond_count = smile.count("=")
    doublebonds.append(doublebond_count)

In [ ]:
for smile in smile_list: 
    mol = Chem.MolFromSmiles(smile)

In [13]:
df_test['hydrogen'] = hydrogen
df_test['carbon'] = carbon
df_test['oxygen'] = oxygen
df_test['nitrogen'] = nitrogen
df_test['silicon'] = silicon
df_test['selenium'] = selenium
df_test['doublebonds'] = doublebonds

ValueError: Length of values does not match length of index

In [54]:
#Drop the 'smiles' column
#df_test = df_test.drop(['smiles'],axis = 1)
vals = df_test.values
X_train = vals[:test_idx]
X_test = vals[test_idx:len(vals)-7]
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape

Train features: (1000000, 7)
Train gap: (1000000,)
Test features: (824230, 7)


In [55]:
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred = LR.predict(X_test)

In [56]:
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_pred = RF.predict(X_test)

In [18]:
BAYES = linear_model.BayesianRidge()
BAYES.fit(X_train, Y_train)
BAYES_pred = BAYES.predict(X_test)

In [49]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [57]:
write_to_file("feat1.csv", LR_pred)
write_to_file("feat2.csv", RF_pred)

In [ ]:
RMSE = np.sqrt(sum(($$INSERT OUTPUT HERE$$-X_test)**2))/824230